In [7]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm

class ImprovedMLP(nn.Module):
    def __init__(self, dropout=0.2, use_residual=True, activation='leaky_relu'):
        super().__init__()
        self.use_residual = use_residual
        
        # Choose activation function
        if activation == 'leaky_relu':
            self.act = nn.LeakyReLU(0.1)
        elif activation == 'gelu':
            self.act = nn.GELU()
        else:
            self.act = nn.ReLU()
            
        # Layer blocks
        self.block1 = nn.Sequential(
            nn.Linear(16, 128),
            nn.BatchNorm1d(128),
            self.act,
            nn.Dropout(dropout)
        )
        
        self.block2 = nn.Sequential(
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            self.act,
            nn.Dropout(dropout)
        )
        
        self.block3 = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            self.act
        )
        
        self.block4 = nn.Sequential(
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            self.act
        )
        
        # Output layer - linear output for flexibility
        self.output = nn.Linear(32, 16)
        
    def forward(self, x):
        b1 = self.block1(x)
        b2 = self.block2(b1)
        
        # Residual connection if dimensions match
        if self.use_residual:
            b2 = b2 + b1
            
        b3 = self.block3(b2)
        b4 = self.block4(b3)
        out = self.output(b4)
        return out

# Wider network architecture for ensemble diversity
class WiderMLP(nn.Module):
    def __init__(self, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(16, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.Linear(64, 16)
        )

    def forward(self, x):
        return self.net(x)

# Deeper network architecture for ensemble diversity
class DeeperMLP(nn.Module):
    def __init__(self, dropout=0.25):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(16, 64),
            nn.LayerNorm(64),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(64, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(128, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.LayerNorm(32),
            nn.GELU(),
            nn.Linear(32, 16)
        )

    def forward(self, x):
        return self.net(x)


def create_model_architecture(model_type, dropout=0.3):
    if model_type == 'improved':
        return ImprovedMLP(dropout=dropout, use_residual=True, activation='leaky_relu')
    elif model_type == 'wider':
        return WiderMLP(dropout=dropout)
    elif model_type == 'deeper':
        return DeeperMLP(dropout=dropout)
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    


def weighted_ensemble_predict(models, weights, x):
    predictions = []
    for i, model in enumerate(models):
        with torch.no_grad():
            pred = model(x)
        predictions.append(pred * weights[i])
    return torch.stack(predictions).sum(0)

In [11]:
import torch
import pandas as pd
import numpy as np

# === Provided input vector ===
input_vector = [
    2.163393, 2.2853286, 2.8403513, 2.5463586,
    1.5416491, 2.531632, 3.3025694, 2.878814,
    3.399083, 2.611855, 2.9322157, 3.363758,
    1.0, 1.8033065, 2.7794342, 3.1031551
]

# === Load normalization stats ===
df = pd.read_csv('/Users/danielmartinezvillegas/Developer/master-ds/✨TDG/audio_pipeline/data/dataset_jose_fwod_vector.csv')
means, stds = [], []
for i in range(16):
    col = f'x_{i}'
    mean, std = df[col].mean(), df[col].std()
    means.append(mean)
    stds.append(std)

# === Normalize input vector ===
input_norm = [
    (input_vector[i] - means[i]) / stds[i] if stds[i] > 0 else input_vector[i]
    for i in range(16)
]
input_tensor = torch.tensor(input_norm, dtype=torch.float32).unsqueeze(0)

# === Load model checkpoint ===
checkpoint_path = "/Users/danielmartinezvillegas/Developer/master-ds/✨TDG/audio_pipeline/test_with_jose_vector/improved_ensemble_model.pt"
checkpoint = torch.load(checkpoint_path)

saved_states = checkpoint['models']
model_weights = checkpoint['weights']

# === Model definitions (reuse your existing definitions) ===
# [Paste your ImprovedMLP, WiderMLP, DeeperMLP, and create_model_architecture here]

# === Rebuild models and set to eval ===
model_types = ['improved', 'wider', 'deeper']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = []

for i, state in enumerate(saved_states):
    model_type = model_types[i % len(model_types)]
    model = create_model_architecture(model_type)
    model.load_state_dict(state)
    model.to(device)
    model.eval()
    models.append(model)

# === Predict ===
input_tensor = input_tensor.to(device)
with torch.no_grad():
    prediction = weighted_ensemble_predict(models, model_weights, input_tensor)

# === Show prediction ===
predicted_vector = prediction.cpu().numpy().flatten()
print("✅ Prediction output:")
print(predicted_vector)


✅ Prediction output:
[0.22691989 0.43765318 0.23921728 0.69033504 0.19941857 0.26326373
 0.3651873  0.43860912 0.20420183 0.62177944 0.28727233 0.6173732
 0.2112942  0.39402276 0.3007992  0.61620724]


/var/folders/b9/t4lk8c451h5dmb3m5570wjvc0000gn/T/ipykernel_77522/4005181687.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [18]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# === Load model checkpoint ===
checkpoint_path = "/Users/danielmartinezvillegas/Developer/master-ds/✨TDG/audio_pipeline/test_with_jose_vector/improved_ensemble_model.pt"
checkpoint = torch.load(checkpoint_path)

saved_states = checkpoint['models']
model_weights = checkpoint['weights']

# === Model definitions must be available in your script ===
# Make sure ImprovedMLP, WiderMLP, DeeperMLP, and create_model_architecture() are defined above this point

# === Rebuild ensemble ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_types = ['improved', 'wider', 'deeper']
models = []

for i, state in enumerate(saved_states):
    model_type = model_types[i % len(model_types)]
    model = create_model_architecture(model_type)
    model.load_state_dict(state)
    model.to(device)
    model.eval()
    models.append(model)

# === Load prediction dataset ===
df = pd.read_csv('/Users/danielmartinezvillegas/Developer/master-ds/✨TDG/audio_pipeline/data/predicted_vectors_jose.csv')

# === Load original dataset to get normalization stats ===
df_stats = pd.read_csv('/Users/danielmartinezvillegas/Developer/master-ds/✨TDG/audio_pipeline/data/dataset_jose_fwod_vector.csv')
means, stds = [], []
for i in range(16):
    col = f'x_{i}'
    m, s = df_stats[col].mean(), df_stats[col].std()
    means.append(m)
    stds.append(s)

# === Normalize input vectors y_0 to y_15 ===
X_raw = df[[f'y_{i}' for i in range(16)]].copy()
X_norm = X_raw.copy()
for i in range(16):
    if stds[i] > 0:
        X_norm[f'y_{i}'] = (X_raw[f'y_{i}'] - means[i]) / stds[i]

X_tensor = torch.tensor(X_norm.values, dtype=torch.float32).to(device)

# === Predict in batches ===
batch_size = 64
predictions = []
with torch.no_grad():
    for i in range(0, len(X_tensor), batch_size):
        batch = X_tensor[i:i+batch_size]
        pred = weighted_ensemble_predict(models, model_weights, batch)
        predictions.append(pred.cpu().numpy())

# === Combine predictions ===
pred_matrix = np.vstack(predictions)
pred_df = pd.DataFrame(pred_matrix, columns=[f'step_{i}' for i in range(16)])

# === Compute per-row MSE between predicted and ground truth ===
true_y = df[[f'y_{i}' for i in range(16)]].values
mse_values = np.mean((pred_matrix - true_y)**2, axis=1)
df['mse'] = mse_values

# === Merge predictions into original dataframe ===
df_final = pd.concat([df.reset_index(drop=True), pred_df], axis=1)

# === Reorder columns as requested ===
base_cols = [
    "sequence", "drummer", "session", "id", "midi_filename", "audio_filename"
]
y_cols = [f"y_{i}" for i in range(16)]
spectrum_cols = [f"{band}_{i}" for i in range(16) for band in ["low", "mid", "high"] if f"{band}_{i}" in df.columns]
meta_cols = ["style", "bpm", "split"]
step_cols = [f"step_{i}" for i in range(16)]
final_cols = base_cols + y_cols + spectrum_cols + meta_cols + step_cols + ["mse"]

# Ensure all columns exist in df_final before reordering
existing_cols = [col for col in final_cols if col in df_final.columns]
df_final = df_final[existing_cols]

# === Save to file ===
output_path = "/Users/danielmartinezvillegas/Developer/master-ds/✨TDG/audio_pipeline/data/predicted_with_steps.csv"
df_final.to_csv(output_path, index=False)

print(f"✅ Predictions and MSE saved to: {output_path}")


/var/folders/b9/t4lk8c451h5dmb3m5570wjvc0000gn/T/ipykernel_77522/2553258028.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


✅ Predictions and MSE saved to: /Users/danielmartinezvillegas/Developer/master-ds/✨TDG/audio_pipeline/data/predicted_with_steps.csv
